# Coding board:

In [8]:
rows='ABCDEFGHI'
digits='123456789'
cols= digits
def cross(a,b):
    return[s+t for s in a for t in b]

In [9]:
boxes=cross(rows,cols)
squares=boxes

In [10]:
row_units=[cross(r,cols) for r in rows] #a1,a2,a3......,a9
column_units=[cross(rows,s) for s in cols]
square_units=[cross(rs,cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

In [11]:
unitlist=row_units+column_units+square_units

In [12]:
units = dict((s, [u for u in unitlist if s in u]) 
             for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)

In [17]:
peers=dict(peers)

# Adding values to board:

In [6]:
def grid_values(grid):
  
    values = []
    all_digits = '123456789'
    for c in grid:
        if c == '.':
            values.append(all_digits)
        elif c in all_digits:
            values.append(c)
    assert len(values) == 81
    return dict(zip(boxes, values))

# Elimination first strategy:

In [7]:
 def eliminate(values):
    solved_values = [box for box in values.keys() if len(values[box]) == 1]  #print keys that have value
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values



# Second strategy

In [8]:
def only_choice(values):
    " 
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

# Constraint Propagation


#     Reducing puzzle

In [9]:
def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        # Use the Eliminate Strategy
        values = eliminate(values)
        # Use the Only Choice Strategy
        values = only_choice(values)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [10]:
def search(values):
    "Using depth-first search and propagation, try all possible values."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return dict(attempt)
        


In [11]:
def solve_sudoku_awad(grid) :
    values = grid_values(grid)
    w=search(values)
    print(w)
    return w
    


In [12]:
import sys, os, random, pygame
sys.path.append(os.path.join("objects"))
import SudokuSquare
from utils import *
from GameResources import *


def play(values, result, history):
    assignments = reconstruct(result, history)
    pygame.init()

    size = width, height = 700, 700
    screen = pygame.display.set_mode(size)

    background_image = pygame.image.load("sudoku-board-bare.jpg").convert()

    clock = pygame.time.Clock()

    while True:
        pygame.event.pump()
        theSquares = []
        initXLoc = 0
        initYLoc = 0
        startX, startY, editable, number = 0, 0, "N", 0
        for y in range(9):
            for x in range(9):
                if x in (0, 1, 2):  startX = (x * 57) + 38
                if x in (3, 4, 5):  startX = (x * 57) + 99
                if x in (6, 7, 8):  startX = (x * 57) + 159

                if y in (0, 1, 2):  startY = (y * 57) + 35
                if y in (3, 4, 5):  startY = (y * 57) + 100
                if y in (6, 7, 8):  startY = (y * 57) + 165
                string_number = values[rows[y] + cols[x]]
                if len(string_number) > 1 or string_number == '' or string_number == '.':
                    number = None
                else:
                    number = int(string_number)
                theSquares.append(SudokuSquare.SudokuSquare(number, startX, startY, editable, x, y))

        screen.blit(background_image, (0, 0))
        for num in theSquares:
            num.draw()

        pygame.display.flip()
        pygame.display.update()
        clock.tick(5)

        if len(assignments) == 0:
            break
        box, value = assignments.pop()
        values[box] = value

    # leave game showing until closed by user
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()


pygame 2.0.1 (SDL 2.0.14, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [13]:
if __name__ == "__main__":
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    display(grid2values(diag_sudoku_grid))
    result = solve_sudoku_awad(diag_sudoku_grid)
    display(result)

    try:
        import PySudoku
        PySudoku.play(grid2values(diag_sudoku_grid), result, history)

    except SystemExit:
        pass
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')

    2    123456789123456789|123456789123456789123456789|123456789123456789123456789
123456789123456789123456789|123456789123456789    6    |    2    123456789123456789
123456789123456789    1    |123456789123456789123456789|123456789    7    123456789
---------------------------+---------------------------+---------------------------
123456789123456789    6    |123456789123456789    8    |123456789123456789123456789
    3    123456789123456789|123456789    9    123456789|123456789123456789    7    
123456789123456789123456789|    6    123456789123456789|    4    123456789123456789
---------------------------+---------------------------+---------------------------
123456789    4    123456789|123456789123456789123456789|    8    123456789123456789
123456789123456789    5    |    2    123456789123456789|123456789123456789123456789
123456789123456789123456789|123456789123456789123456789|123456789123456789    3    

{'A1': '2', 'A2': '3', 'A3': '9', 'A4': '8', 'A5': '7', 'A6': '4', 'A7': '1